In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.python.tools import freeze_graph
from tensorflow.python.tools import optimize_for_inference_lib
import csv
import math

In [2]:
#Reading data file
data_circle = pd.read_csv('circle.csv', sep=',',delimiter=None, header=None)
data_cross = pd.read_csv('cross.csv', sep=',', delimiter=None, header=None)
data_square = pd.read_csv('square.csv', sep=',', delimiter=None, header=None)
data_triangle = pd.read_csv('triangle.csv', sep=',', delimiter=None, header=None)

#Join data and shuffle randomly
data_combined = pd.concat([data_circle, data_cross, data_square, data_triangle])
data_combined.to_csv( index=False )
data_combined = data_combined.reset_index()
data_suffled = data_combined.sample(frac=1).reset_index(drop=True)
data_suffled = data_suffled.drop('index',1)

#Write csv back to the file
#data_suffled.to_csv("combined_data.csv", sep = ',')

#Processing data as X_all and Y_all
X_all = data_suffled.iloc[:,0:784]
Y_all = data_suffled.iloc[:,784]


#Converting Y_all to hot vectors
#converter = lambda x: ord(x)-ord('A')
#Y_all = list(map(converter,Y_all.values.T.tolist()))
Y_all = list(Y_all.values.T.tolist())
Y_all = pd.get_dummies(Y_all).values


X_all = pd.DataFrame(X_all)
Y_all = pd.DataFrame(Y_all)
#print(X_all)
#print(Y_all)


In [3]:
print(X_all.shape)
print(Y_all.shape)

(409, 784)
(409, 4)


In [4]:
NUM_CLASSES = Y_all.shape[1]
INPUT_SIZE = X_all.shape[1]
BATCH_SIZE = 20
NR_STEP = int(X_all.shape[0]/BATCH_SIZE)
NR_EPOCH = 50

layer_info = [INPUT_SIZE, 1024, 100, 150, 20, NUM_CLASSES]
layer_size = len(layer_info)

logs_path = '/tf_log/'

print(layer_info)

[784, 1024, 100, 150, 20, 4]


In [5]:
def inference_graph(X, layer_info):
    #Define weight, bias and output under different namescope
    layer_input = X
    for i in range(1,len(layer_info)-1):
        with tf.name_scope('layer_'+str(i)):
            W = tf.Variable(tf.truncated_normal([layer_info[i-1],layer_info[i]], 
                                                stddev=1.0/math.sqrt(float(layer_info[i-1]))),
                           name = 'weight_'+str(i))
            tf.summary.histogram('weight_histogram_'+str(i), W)
            b = tf.Variable(tf.zeros([layer_info[i]]),
                           name = 'bias_'+str(i))
            tf.summary.histogram('bias_histogram_'+str(i), b)
            layer_output = tf.nn.relu(tf.matmul(layer_input, W) + b)
            
            #print(layer_output)
            
            layer_input = layer_output
    
    nr_layer = len(layer_info)
    with tf.name_scope('layer_'+str(nr_layer-1)):
        W = tf.Variable(tf.truncated_normal([layer_info[nr_layer-2],layer_info[nr_layer-1]], 
                                            stddev=1.0/math.sqrt(float(layer_info[nr_layer-2]))),
                       name = 'weight_'+str(nr_layer-1))
        tf.summary.histogram('weight_histogram_'+str(nr_layer-1), W)
        b = tf.Variable(tf.zeros([layer_info[nr_layer-1]]),
                        name = 'bias_'+str(nr_layer-1))
        tf.summary.histogram('bias_histogram_'+str(nr_layer-1), b)
        layer_output = tf.matmul(layer_input, W) + b
        softmax_output = tf.nn.softmax(layer_output, name='OUTPUT')

    return layer_output, softmax_output

In [6]:
# Training graph construction

def train_graph(output, logit, Y, learning_rate):
    print(logit.shape)
    with tf.name_scope('cross_entropy'):
        softmax_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=logit, name = 'xentropy')
        loss = tf.reduce_mean(softmax_entropy, name = 'loss')
        tf.summary.scalar('loss', loss)
    
    with tf.name_scope('train'):
        optimizer = tf.train.GradientDescentOptimizer(learning_rate)
        train_op = optimizer.minimize(loss)
        
    with tf.name_scope('accuracy'):
        nr_correct = tf.equal(tf.argmax(Y,1), tf.argmax(output,1))
        accuracy = tf.reduce_mean(tf.cast(nr_correct, tf.float32))
        tf.summary.scalar('accuracy_mean', accuracy)
    
    return loss, train_op, accuracy

In [7]:
#Saving graph to protobuf requires three things:
#1. Saving graph defenition (format: pbtxt)
#2. Saving weights checkpoint (format: chkp)
#3. Freezing graph along with weight (format: pb)

def export_model(input_name, output_name):
    freeze_graph.freeze_graph('out/model_def.pbtxt', None, False, 'out/model_checkpoint.chkp', output_name, 
                             "save/restore_all", "save/Const:0", 'out/model_frozen.pb', True, "")
    input_graph_def = tf.GraphDef()
    with tf.gfile.Open('out/model_frozen.pb', "rb") as f:
        input_graph_def.ParseFromString(f.read())
    output_graph_def = optimize_for_inference_lib.optimize_for_inference(
                        input_graph_def, [input_name], [output_name],tf.float32.as_datatype_enum)
    with tf.gfile.FastGFile('out/optimized_model.pb', "wb") as f:
        f.write(output_graph_def.SerializeToSring())


In [8]:
#Tensorflow full graph construction

model_graph = tf.Graph()
with model_graph.as_default():
    with tf.name_scope('input'):
        X = tf.placeholder(tf.float32, shape = [None, INPUT_SIZE], name = 'X_INPUT')
        Y = tf.placeholder(tf.float32, shape = [None, NUM_CLASSES], name = 'Y_INPUT')

    logits, output = inference_graph(X, layer_info)
    loss, train_op, accuracy = train_graph(output, logits, Y, 0.01)

    merged_summary_op = tf.summary.merge_all()

    init = tf.initialize_all_variables()


(?, 4)
Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [9]:
#Training model

with tf.Session(graph = model_graph) as sess:
    
    sess.run(init)
    summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
    #To save graph defenition
    tf.train.write_graph(sess.graph_def, 'out','model_def.pbtxt' , True)
    
    for epoch in range(0, NR_EPOCH):
        for step in range(0, NR_STEP):
            feed_X = X_all.iloc[step*BATCH_SIZE:(step+1)*BATCH_SIZE, :]
            feed_Y = Y_all.iloc[step*BATCH_SIZE:(step+1)*BATCH_SIZE, :]
            _, step_accuracy, step_loss, summary = sess.run([train_op, accuracy, loss, merged_summary_op],
                                                  feed_dict={X:feed_X, Y:feed_Y})
            summary_writer.add_summary(summary, epoch)
            
            if step % 1000 == 0:
                print('Epoch= %d, step= %d, accuracy= %.2f loss= %.2f' % (epoch, step, step_accuracy, step_loss))
    #For saving the trained model's weight in checkpoint
    saver = tf.train.Saver()
    saver.save(sess, 'out/' + 'model_checkpoint.chkp')

Epoch= 0, step= 0, accuracy= 0.20 loss= 1.38
Epoch= 1, step= 0, accuracy= 0.20 loss= 1.31
Epoch= 2, step= 0, accuracy= 0.40 loss= 1.23
Epoch= 3, step= 0, accuracy= 0.65 loss= 1.14
Epoch= 4, step= 0, accuracy= 0.70 loss= 1.03
Epoch= 5, step= 0, accuracy= 0.70 loss= 0.91
Epoch= 6, step= 0, accuracy= 0.75 loss= 0.79
Epoch= 7, step= 0, accuracy= 0.85 loss= 0.66
Epoch= 8, step= 0, accuracy= 0.90 loss= 0.54
Epoch= 9, step= 0, accuracy= 0.90 loss= 0.46
Epoch= 10, step= 0, accuracy= 0.90 loss= 0.41
Epoch= 11, step= 0, accuracy= 0.90 loss= 0.38
Epoch= 12, step= 0, accuracy= 0.90 loss= 0.34
Epoch= 13, step= 0, accuracy= 0.90 loss= 0.32
Epoch= 14, step= 0, accuracy= 0.90 loss= 0.28
Epoch= 15, step= 0, accuracy= 1.00 loss= 0.25
Epoch= 16, step= 0, accuracy= 1.00 loss= 0.22
Epoch= 17, step= 0, accuracy= 1.00 loss= 0.20
Epoch= 18, step= 0, accuracy= 1.00 loss= 0.18
Epoch= 19, step= 0, accuracy= 1.00 loss= 0.17
Epoch= 20, step= 0, accuracy= 1.00 loss= 0.15
Epoch= 21, step= 0, accuracy= 1.00 loss= 0.1

In [12]:
export_model(['X_INPUT'], 'OUTPUT')

INFO:tensorflow:Restoring parameters from out/model_checkpoint.chkp


AttributeError: 'list' object has no attribute 'split'